# Code ver. 1
- __Date__: Jun 04, 2020
- __Author__: Yeji Soh
- __Description__: This code is to pick relevant census data filtered by user's geographical (latitude & longitude) data. Note that for now the code only works for the area of Territories only, so please make sure to enter in any latitude and longitude within Territories. Further things to develop is written in **NOTE** section below. Please feel free to add more stuffs that should be consdiered to that section.

### Import Territories dataset

In [1]:
import pandas as pd
import numpy as np

In [64]:
df = pd.read_csv('TERRITORIES.csv')

In [3]:
df.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM: Profile of Dissemination Areas (2247),Member ID: Profile of Dissemination Areas (2247),Notes: Profile of Dissemination Areas (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2016",1.0,1.0,35151728,...,...
1,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2011",2.0,2.0,33476688,...,...
2,2016,1,0,Canada,4.0,5.1,20000,1,"Population percentage change, 2011 to 2016",3.0,NaN,5.0,...,...
3,2016,1,0,Canada,4.0,5.1,20000,1,Total private dwellings,4.0,3.0,15412443,...,...
4,2016,1,0,Canada,4.0,5.1,20000,1,Private dwellings occupied by usual residents,5.0,4.0,14072079,...,...


### Test Lat & Long and reverse-geocode

In [4]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [7]:
locator = Nominatim(user_agent="myGeocoder")

In [28]:
coord="52.675885, -113.594679"
location = locator.reverse(coord)
location.raw

{'place_id': 7424509,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 101675136,
 'lat': '52.6760421',
 'lon': '-113.5954945',
 'display_name': '6210, 48 Avenue, Ponoka, Alberta, T4J 1K3, Canada',
 'address': {'house_number': '6210',
  'road': '48 Avenue',
  'town': 'Ponoka',
  'state': 'Alberta',
  'postcode': 'T4J 1K3',
  'country': 'Canada',
  'country_code': 'ca'},
 'boundingbox': ['52.6759921', '52.6760921', '-113.5955445', '-113.5954445']}

In [29]:
if location.raw['address']['town' or 'city'] in location[0]:
    print("This place is in",location.raw['address']['town' or 'city'])

This place is in Ponoka


### Code to filter out data by user's location

__01. See the GEO_Name in dataset__

In [10]:
# See how many geo in data
df2 = df.drop_duplicates(subset = ["GEO_NAME"])

In [11]:
df2 = df2[['GEO_NAME']]
df2.reset_index(drop=True, inplace=True)

In [12]:
df2 = df2.sort_values(by='GEO_NAME', ascending=False)
df2.reset_index(drop=True, inplace=True)

In [13]:
df2

,GEO_NAME
0,"Yukon, Unorganized"
1,Yukon
2,Yellowknife
3,Wrigley
4,"Whitehorse, Unorganized"
...,...
243,60010122
244,60010121
245,60010120
246,60010119


__02. Get latitude & Longitude from User and geographical location accordingly__

In [98]:
lat = input("Enter the latitude: ")
print(lat)
lng = input("Enter the longitude: ")
print(lng)

Enter the latitude: 62.459594
62.459594
Enter the longitude: -114.412841
-114.412841


In [99]:
coord1=lat+","+lng

In [100]:
location = locator.reverse(coord1)
location.raw

{'place_id': 8159189,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 104193767,
 'lat': '62.45937108021979',
 'lon': '-114.41263414994641',
 'display_name': '354, Old Airport Road, New Town, Yellowknife, North Slave Region, Northwest Territories, X1A 3T4, Canada',
 'address': {'house_number': '354',
  'road': 'Old Airport Road',
  'suburb': 'New Town',
  'city': 'Yellowknife',
  'county': 'North Slave Region',
  'state': 'Northwest Territories',
  'postcode': 'X1A 3T4',
  'country': 'Canada',
  'country_code': 'ca'},
 'boundingbox': ['62.45932108022',
  '62.45942108022',
  '-114.41268414995',
  '-114.41258414995']}

In [96]:
location.raw['address']['city' or 'town']

'Yellowknife'

__03. Check the user geo info is found in dataset <br>
This part would be replaced later with the code checking if the coordinate given is found in boundary files__

In [97]:
if location.raw['address']['city' or 'town'] in df2.values:
    print("This place is in dataset.")
else:
    print("This place is NOT in dataset.")

This place is in dataset.


### Filter the information corresponding to user's location

In [65]:
df.rename(columns={"DIM: Profile of Dissemination Areas (2247)": "Data", "Dim: Sex (3): Member ID: [1]: Total - Sex": "Total Gender", "Dim: Sex (3): Member ID: [2]: Male":"Male","Dim: Sex (3): Member ID: [3]: Female":"Female"}, inplace=True)
df.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,Data,Member ID: Profile of Dissemination Areas (2247),Notes: Profile of Dissemination Areas (2247),Total gender,Male,Female
0,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2016",1.0,1.0,35151728,...,...
1,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2011",2.0,2.0,33476688,...,...
2,2016,1,0,Canada,4.0,5.1,20000,1,"Population percentage change, 2011 to 2016",3.0,NaN,5.0,...,...
3,2016,1,0,Canada,4.0,5.1,20000,1,Total private dwellings,4.0,3.0,15412443,...,...
4,2016,1,0,Canada,4.0,5.1,20000,1,Private dwellings occupied by usual residents,5.0,4.0,14072079,...,...


In [87]:
df_filtered = df[df['GEO_NAME'].str.contains(location.raw['address']['city' or 'town'], na=False)]
df_filtered.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,Data,Member ID: Profile of Dissemination Areas (2247),Notes: Profile of Dissemination Areas (2247),Total gender,Male,Female
478611,2016,6106023,3,Yellowknife,6.4,8.9,101,6106023,"Population, 2016",1.0,1.0,19569,...,...
478612,2016,6106023,3,Yellowknife,6.4,8.9,101,6106023,"Population, 2011",2.0,2.0,19234,...,...
478613,2016,6106023,3,Yellowknife,6.4,8.9,101,6106023,"Population percentage change, 2011 to 2016",3.0,NaN,1.7,...,...
478614,2016,6106023,3,Yellowknife,6.4,8.9,101,6106023,Total private dwellings,4.0,3.0,7758,...,...
478615,2016,6106023,3,Yellowknife,6.4,8.9,101,6106023,Private dwellings occupied by usual residents,5.0,4.0,7131,...,...


In [88]:
df_filtered = df_filtered.drop(columns=['GNR','GNR_LF','DATA_QUALITY_FLAG','ALT_GEO_CODE','GEO_LEVEL','Member ID: Profile of Dissemination Areas (2247)','Notes: Profile of Dissemination Areas (2247)'], axis=1)

In [89]:
df_filtered.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_NAME,Data,Total gender,Male,Female
478611,2016,6106023,Yellowknife,"Population, 2016",19569,...,...
478612,2016,6106023,Yellowknife,"Population, 2011",19234,...,...
478613,2016,6106023,Yellowknife,"Population percentage change, 2011 to 2016",1.7,...,...
478614,2016,6106023,Yellowknife,Total private dwellings,7758,...,...
478615,2016,6106023,Yellowknife,Private dwellings occupied by usual residents,7131,...,...


In [91]:
df_filtered[df['Data'] == 'Population, 2016']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,CENSUS_YEAR,GEO_CODE (POR),GEO_NAME,Data,Total gender,Male,Female
478611,2016,6106023,Yellowknife,"Population, 2016",19569,...,...


#### ** NOTE**
- Should replace the ones of which GEO_NAME is in number format with the name, once this info can be brought from shape file.
- Need to generalize the dataset, not just Territories, but all the provinces included.
- The code for now is just when single user info is given, need to make it work for when multiple user lat&long are given.
- Need to discuss further on which parameters we should consider.
- See if user's lat&long is within polygon lat&long boundaries: Challenge - lat&long doesnt' exist in census data, only the geo_name.